In [14]:
import gzip
import pandas as pd
from collections import defaultdict
import math
import scipy.optimize
import numpy as np
import string
import random
from sklearn import linear_model
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem.porter import *

In [2]:
data_raw_interactions = pd.read_csv("RAW_interactions.csv")


In [4]:
data = data_raw_interactions.sample(50000, random_state = 42)
del data_raw_interactions

In [6]:
data.iloc[0,:]

user_id                                                 764278
recipe_id                                                32614
date                                                2012-12-22
rating                                                       5
review       I made these last night for a Christmas caroli...
Name: 592612, dtype: object

In [12]:
data['sentiment'] = data.apply(lambda row: 1 if row.rating in [5,4] else 0, axis=1)
data['sentiment'] = data.apply(lambda row: -1 if row.rating in [1,2] else row.sentiment, axis=1)

In [32]:
data['review'] = data['review'].astype(str)

In [33]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for i, d in data.iterrows():
    r = ''.join([c for c in d['review'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

len(wordCount)

43344

In [35]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]

In [37]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [34]:
def feature(datum, words, wordId, tolower=True, removePunct=True):
    feat = [0]*len(words)
    r = datum['review']
    if tolower:
        r = r.lower()
    if removePunct:
        r = ''.join([c for c in r if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat